<a href="https://colab.research.google.com/github/ForestHouse2316/KoreanOCR-for-Web/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>